In [10]:
from groq import Groq
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
from google.colab import files

# Initialize Groq
client = Groq(api_key="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")

# Load BLIP model (vision → text)
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# --- Vision Agent (BLIP) ---
def vision_agent(image_path: str) -> str:
    image = Image.open(image_path).convert("RGB")
    inputs = processor(images=image, return_tensors="pt")
    out = model.generate(**inputs)
    description = processor.decode(out[0], skip_special_tokens=True)
    return f"Scene description: {description}"

# --- Creative Agent (Groq) ---
def creative_agent(scene_context: str) -> str:
    completion = client.chat.completions.create(
        model="gemma2-9b-it",
        messages=[
            {
                "role": "system",
                "content": "You are SnapCaption, an AI social media caption generator. "
                           "Take a short description of a photo and create 1–2 catchy, "
                           "fun captions under 15 words with emojis."
            },
            {"role": "user", "content": scene_context}
        ],
        temperature=0.9,
        max_completion_tokens=50,
        top_p=1,
        stream=False
    )
    return completion.choices[0].message.content.strip()

# --- Orchestrator ---
def generate_caption(image_path: str):
    scene = vision_agent(image_path)
    caption = creative_agent(scene)
    return scene, caption

# --- Upload flow ---
print("📎 Please upload an image file (jpg/png)...")
uploaded = files.upload()

# Loop through uploaded files
for fname in uploaded.keys():
    print(f"\n✅ File received: {fname}")
    scene, caption = generate_caption(fname)
    print("🔍 Vision Agent Output:", scene)
    print("✨ Generated Caption:", caption)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

📎 Please upload an image file (jpg/png)...


Saving Wallpaper_Photoholic_1o-12.jpg to Wallpaper_Photoholic_1o-12.jpg

✅ File received: Wallpaper_Photoholic_1o-12.jpg
🔍 Vision Agent Output: Scene description: a large statue of a hindu deity
✨ Generated Caption: Here are some captions for a photo of a Hindu deity statue:

* ** 🙏 Majestic and divine!  #HinduDeity  **
* **  Feeling blessed in the presence of greatness. 🙏✨  ** 


Let me know if
